In [32]:
import pandas as pd
from dateutil import parser

In [45]:
df = pd.read_csv('fashionnova_sept1_2021_to_mar25_2022.csv')

In [46]:
df
df['created_at'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d')
df.head()

,tweet_id,created_at,text,author_id
0,1507490223764254727,2022-03-25,@FashionNova there has to be another way to co...,408065974
1,1507481067279814656,2022-03-25,@BWay36 @MrComplaintBox @LaserShipAssist @Lase...,240972762
2,1507480310262505472,2022-03-25,I hate @FashionNova never tf again,1639514174
3,1507474208242257923,2022-03-25,So @FashionNova DMd me to help resolve my issu...,1908290126
4,1507469126842859525,2022-03-25,How can I stop back in stock emails from @Fash...,1035591222772080642


In [47]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

import re
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\eurydice\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eurydice\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [48]:
sentence = df['text'][8]
print(sentence)
sid.polarity_scores(sentence)['compound']

@EvaAndressaOf @FashionNova Who's  bored  here too?


-0.2732

In [49]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet

In [50]:
df['tweet_clean'] = df['text'].apply(cleaner)

In [51]:
word_dict = {'manipulate':-1,'manipulative':-1,'cancel':-1,'cancelled':-1,'cancel culture':0.4,'liar':-1}

sid.lexicon.update(word_dict)
list1 = []
for i in df['tweet_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

In [52]:
df['sentiment'] = pd.Series(list1)
def sentiment_category(sentiment):
    label = ''
    if(sentiment>0):
        label = 'positive'
    elif(sentiment == 0):
        label = 'neutral'
    else:
        label = 'negative'
    return(label)

df['sentiment_category'] = df['sentiment'].apply(sentiment_category)

In [53]:
df = df[['text','created_at','tweet_id','sentiment','sentiment_category']]
df.head()

,text,created_at,tweet_id,sentiment,sentiment_category
0,@FashionNova there has to be another way to co...,2022-03-25,1507490223764254727,0.4019,positive
1,@BWay36 @MrComplaintBox @LaserShipAssist @Lase...,2022-03-25,1507481067279814656,-0.4215,negative
2,I hate @FashionNova never tf again,2022-03-25,1507480310262505472,-0.5719,negative
3,So @FashionNova DMd me to help resolve my issu...,2022-03-25,1507474208242257923,0.6738,positive
4,How can I stop back in stock emails from @Fash...,2022-03-25,1507469126842859525,-0.5994,negative


In [54]:
neg = df[df['sentiment_category']=='negative']
neg = neg.groupby(['created_at'],as_index=False).count()

pos = df[df['sentiment_category']=='positive']
pos = pos.groupby(['created_at'],as_index=False).count()

pos = pos[['created_at','tweet_id']]
neg = neg[['created_at','tweet_id']]

In [55]:
import plotly.graph_objs as go
fig = go.Figure()
for col in pos.columns:
    fig.add_trace(go.Scatter(x=pos['created_at'], y=pos['tweet_id'],
                             name = col,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True,
                             line_color='green'
                             )
                 )
for col in neg.columns:
    fig.add_trace(go.Scatter(x=neg['created_at'], y=neg['tweet_id'],
                             name = col,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True,
                             line_color='red'
                             )
                 )
fig.show()

In [63]:
# filter the df to only capture Tweets from the start of mid sept to end of oct
newdf = df[(df['created_at']>='2021-10-01') & (df['created_at']<='2021-10-10')]
neg = newdf[newdf['sentiment_category']=='negative']
neg = neg.groupby(['created_at'],as_index=False).count()
pos = newdf[newdf['sentiment_category']=='positive']
pos = pos.groupby(['created_at'],as_index=False).count()

pos = pos[['created_at','tweet_id']]
neg = neg[['created_at','tweet_id']]

In [64]:
import plotly.graph_objs as go
fig = go.Figure()
for col in pos.columns:
    fig.add_trace(go.Scatter(x=pos['created_at'], y=pos['tweet_id'],
                             name = col,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True,
                             line_color='green'
                             )
                 )
for col in neg.columns:
    fig.add_trace(go.Scatter(x=neg['created_at'], y=neg['tweet_id'],
                             name = col,
                             mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True,
                             line_color='red'
                             )
                 )
fig.show()

In [69]:
oct4_df = df[(df['created_at'] > '2021-10-03') & (df['created_at'] < '2021-10-05')]
oct4_df['text'].head()

14482    The devil works hard but @FashionNova works ha...
14483                   @_Lolobe4 @FashionNova Dreamy!! ❤⚘
14484    Fact 26: @fashionnova I’m over here working ov...
14485    @FashionNova This would be amazing! I would gi...
14486    @FashionNova it’s not the time !! I’m broke ri...
Name: text, dtype: object